<a href="https://colab.research.google.com/github/varshithr/ner/blob/main/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLTK Method - Test out the accuracy and approach

In [ ]:
!pip install nltk

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag



In [ ]:
# Load Named Entity Recognition model
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def is_person_name(text):
    tokens = word_tokenize(text)
    tagged = nltk.pos_tag(tokens)
    named_entities = nltk.ne_chunk(tagged)
    for subtree in named_entities.subtrees():
        print(subtree.label(), dir(subtree))
        if subtree.label() == 'PERSON':
            return True
    return False

# Test the function
print(is_person_name("John Smith"))  # True
print(is_person_name("New York"))  # False

S ['__add__', '__class__', '__class_getitem__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__radd__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_frozen_class', '_get_node', '_label', '_parse_error', '_pformat_flat', '_repr_svg_', '_set_node', 'append', 'chomsky_normal_form', 'clear', 'collapse_unary', 'convert', 'copy', 'count', 'draw', 'extend', 'flatten', 'freeze', 'fromlist', 'fromstring', 'height', 'index', 'insert', 'label', 'leaf_treeposition', 'leaves', 'node', 'pformat', 'pformat_latex_qtree', 'pop', 'pos', 'pprint', 'pretty_print', 'productions', 'remove', 'reve

In [ ]:
text = "Hyderabad"

In [ ]:
tokens = word_tokenize(text)
tagged = nltk.pos_tag(tokens)
named_entities = nltk.ne_chunk(tagged)
for subtree in named_entities.subtrees():
    print(subtree.label(), subtree.leaves())

S [('Hyderabad', 'NN')]
GPE [('Hyderabad', 'NN')]


In [ ]:
import requests
import csv
import time
from typing import Dict, Any

## Fetch firm and person information FINRA without hitting rate limits for test data creation (USE VPN)

In [ ]:
def fetch_all_rows(base_url: str, params: Dict[str, Any], nrows: int = 100) -> list:
    all_data = []
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    resp = response.json()
    from pprint import pprint
    # pprint(resp)
    data = response.json()['hits']
    total_hits = resp['hits']['total']
    pages_needed = (total_hits // nrows) + (1 if total_hits % nrows > 0 else 0)

    for page in range(min(pages_needed, 100)):
        print(f"Fetching page {page + 1} of {pages_needed}")
        params["start"] = page * nrows
        try:
            response = requests.get(base_url, params=params)
            response.raise_for_status()
            data = response.json()['hits']['hits']
            all_data.extend(data)
        except:
            # print(f"Request failed: {e}")
            break

        time.sleep(1)  # Avoid hitting rate limits

    return all_data

base_url = "https://api.brokercheck.finra.org/search/individual"
params_template = {
        "includePrevious": True,
        "hl": True,
        "nrows": 100,
        "r": 25,
        "sort": "score+desc",
        "wt": "json"
    }
# total_hits = 268210

all_alphabet_data = []
for letter in 'abcdefghijklmnopqrstuvwxyz':
    print(f"Fetching data for initial '{letter}'")
    params = params_template.copy()
    params["query"] = letter
    data = fetch_all_rows(base_url, params)
    all_alphabet_data.extend(data)

Fetching data for initial 'a'
Fetching page 1 of 2683
Fetching page 2 of 2683
Fetching page 3 of 2683
Fetching page 4 of 2683
Fetching page 5 of 2683
Fetching page 6 of 2683
Fetching page 7 of 2683
Fetching page 8 of 2683
Fetching page 9 of 2683
Fetching page 10 of 2683
Fetching page 11 of 2683
Fetching page 12 of 2683
Fetching page 13 of 2683
Fetching page 14 of 2683
Fetching page 15 of 2683
Fetching page 16 of 2683
Fetching page 17 of 2683
Fetching page 18 of 2683
Fetching page 19 of 2683
Fetching page 20 of 2683
Fetching page 21 of 2683
Fetching page 22 of 2683
Fetching page 23 of 2683
Fetching page 24 of 2683
Fetching page 25 of 2683
Fetching page 26 of 2683
Fetching page 27 of 2683
Fetching page 28 of 2683
Fetching page 29 of 2683
Fetching page 30 of 2683
Fetching page 31 of 2683
Fetching page 32 of 2683
Fetching page 33 of 2683
Fetching page 34 of 2683
Fetching page 35 of 2683
Fetching page 36 of 2683
Fetching page 37 of 2683
Fetching page 38 of 2683
Fetching page 39 of 2683
Fetc

### Save to CSV while ignoring nulls in some columns

In [ ]:
def save_to_csv(data: list, filename: str):
    keys = data[0]['_source'].keys()
    with open(filename, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=keys)
        writer.writeheader()
        for item in data:
            row = item['_source']
            writer.writerow({k: v if v is not None else '' for k, v in row.items()})

In [ ]:
save_to_csv(all_alphabet_data, "finra_registered_individuals.csv")

print(f"Saved {len(all_alphabet_data)} rows to finra_registered_individuals.csv")

ValueError: dict contains fields not in fieldnames: 'ind_industry_days'

In [ ]:
l = [{"_source": {"col1": "value", "col2": "value"}, "col3": "value"}, {"_source": {"col1": "value", "col2": "value"}, "col3": "value"}]

In [ ]:
from pprint import pprint
pprint(l)

[{'_source': {'col1': 'value', 'col2': 'value'}, 'col3': 'value'},
 {'_source': {'col1': 'value', 'col2': 'value'}, 'col3': 'value'}]


In [ ]:
l[0].keys(  )

dict_keys(['_source', 'col3'])

In [ ]:
# Flatten the dictionaries and extract _source
flattened_data = [item["_source"] for item in all_alphabet_data]

# Dynamically determine the set of unique keys present across all dictionaries
unique_keys = set(key for d in flattened_data for key in d.keys())

# Write to CSV
filename = "output_inconsistent_fields.csv"
with open(filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=sorted(unique_keys))
    writer.writeheader()
    for row in flattened_data:
        # Use dict.get() to handle missing keys gracefully
        writer.writerow({key: row.get(key, None) for key in unique_keys})

print(f"Data saved to {filename}")

Data saved to output_inconsistent_fields.csv


In [ ]:
!pip install transformers

In [ ]:
!pip install ipywidgets

In [ ]:
!pip install torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

In [ ]:
def ner_tag_transformer(text):
    pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="max")
    return pipe(text)

# from pprint import pprint

In [ ]:
import re

In [ ]:
from pprint import pprint

In [ ]:
capitalize_first_letter_of_every_word(r'"LEIGH BALDWIN & CO., LLC"')

'Leigh Baldwin & CO., Llc'

In [ ]:
pprint(ner_tag_transformer(r'EDWARD JONES'))

[{'end': 12,
  'entity_group': 'ORG',
  'score': 0.54005075,
  'start': 7,
  'word': 'JONES'}]


In [ ]:
pprint(ner_tag_transformer(capitalize_first_letter_of_every_word("LEIGH BALDWIN & CO., LLC")))

[{'end': 5,
  'entity_group': 'ORG',
  'score': 0.746765,
  'start': 0,
  'word': 'Leigh'},
 {'end': 13,
  'entity_group': 'PER',
  'score': 0.51786023,
  'start': 6,
  'word': 'Baldwin'},
 {'end': 19,
  'entity_group': 'ORG',
  'score': 0.61837167,
  'start': 14,
  'word': '& CO.'}]


In [ ]:
def capitalize_first_letter_of_every_word(text, special_words=None):
    if special_words is None:
        special_words = ["LLC", "M.D.", "CEO", "WMG"]  # special words here

    special_words_lower = [word.lower() for word in special_words]
    words = text.split()
    capitalized_words = []

    for word in words:
        if word.lower() in special_words_lower:
            capitalized_words.append(word.upper())
        elif '.' in word:
            capitalized_words.append(word.upper())
        else:
            capitalized_words.append(word.capitalize())

    return ' '.join(capitalized_words)


In [ ]:
pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
"""All the valid aggregation strategies for TokenClassificationPipeline"""

    # NONE = "none"
    # SIMPLE = "simple"
    # FIRST = "first"
    # AVERAGE = "average"
    # MAX = "max"

def extract_ner_info(text):
    text = capitalize_first_letter_of_every_word(text)
    ner_results = pipe(text)

    extracted_info = []

    # conf score > 80 pct
    # filtered_results = [result for result in ner_results if result.get('score') >= 0.8]

    for result in ner_results:
        entity_group = result['entity_group']
        word = result['word']
        score = result['score']
        extracted_info.append((entity_group, word, score))

    return extracted_info

NameError: name 'pipeline' is not defined

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('firm_names.csv')

In [ ]:
df['ner_info'] = df['firm_name'].apply(extract_ner_info) #apply transformer model

NameError: name 'extract_ner_info' is not defined

In [ ]:
df_exploded = df.explode('ner_info') # explode nested list

In [ ]:
df_exploded[['entity_group', 'word', 'score']] = pd.DataFrame(df_exploded['ner_info'].tolist(), index=df_exploded.index)

In [ ]:
df_final = df_exploded.drop(columns=['ner_info'])

In [ ]:
df_final.to_csv('firm_output2.csv', index=False) # save output

# Training starts here

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.

In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
import torch
from datasets import load_dataset

## Labels

In [3]:
labels = ['PER', 'ORG']
label_map = {label: idx for idx, label in enumerate(labels)} #assign numbers to labels

In [4]:
import pandas as pd

In [5]:
org_df= pd.read_csv("firm_names.csv")

In [6]:
org_list = org_df['firm_name'].tolist()

In [7]:
annotated_org_list = [("The company " + org, {"entities": [(14, 14 + len(org), "ORG")]}) for org in org_list]

In [8]:
annotated_org_list[0]

('The company NYLIFE SECURITIES LLC', {'entities': [(14, 35, 'ORG')]})

In [9]:
combined_list = [(sentence, {'entities': [(start, end, 'ORG') for start, end, _ in annotation['entities']]}) for sentence, annotation in annotated_org_list]

In [10]:
combined_list[0]

('The company NYLIFE SECURITIES LLC', {'entities': [(14, 35, 'ORG')]})

In [ ]:
from sklearn.model_selection import train_test_split

def prepare_datasets(combined_list, test_size=0.2):
    # Split the combined list into training and evaluation sets
    train_list, eval_list = train_test_split(combined_list, test_size=test_size, random_state=42)

    def encode_and_label(data_list):
        encodings = []
        labels = []
        for sentence, annotation in data_list:
            encoding = tokenizer(sentence, truncation=True, padding='max_length', max_length=128)
            label = [0] * len(encoding.input_ids)
            for start, end, entity_label in annotation['entities']:
                label[start:end] = [label_map[entity_label]] * (end - start)
            encodings.append(encoding)
            labels.append(label)
        return encodings, labels

    # Prepare training and evaluation datasets
    train_encodings, train_labels = encode_and_label(train_list)
    eval_encodings, eval_labels = encode_and_label(eval_list)

    return train_encodings, train_labels, eval_encodings, eval_labels



In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner", num_labels=len(label_map), ignore_mismatched_sizes=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at Babelscape/wikineural-multilingual-ner and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream t

In [ ]:
train_encodings, train_labels, eval_encodings, eval_labels = prepare_datasets(combined_list)

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!pip install accelerate -U

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',  # Directory to save model checkpoints and training logs
    num_train_epochs=3,  # Number of times to iterate over the entire training dataset
    per_device_train_batch_size=8,  # Batch size per device during training
    per_device_eval_batch_size=32,  # Batch size per device during evaluation
    warmup_steps=500,  # Number of steps for the learning rate to warm up
    weight_decay=0.01,  # Strength of weight decay regularization
    logging_dir='./logs',  # Directory to store TensorBoard logs
    gradient_accumulation_steps=2,  # Accumulate gradients over multiple batches before updating weights
    evaluation_strategy='epoch',  # Evaluate the model at the end of each epoch
    save_strategy='epoch',  # Save a model checkpoint at the end of each epoch
    load_best_model_at_end=True,  # Load the best model based on evaluation metrics at the end of training
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val) for key, val in self.encodings[idx].items()}  # Removed unsqueeze(0)
        item['labels'] = torch.tensor(self.labels[idx])  # Removed unsqueeze(0)
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = CustomDataset(train_encodings, train_labels)
eval_dataset = CustomDataset(eval_encodings, eval_labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

trainer.train()

Epoch,Training Loss,Validation Loss
0,No log,0.013450
2,0.009400,0.001975


TrainOutput(global_step=1440, training_loss=0.00903810891840193, metrics={'train_runtime': 678.8551, 'train_samples_per_second': 33.962, 'train_steps_per_second': 2.121, 'total_flos': 1504677373664256.0, 'train_loss': 0.00903810891840193, 'epoch': 2.9968782518210197})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.0019750508945435286,
 'eval_runtime': 11.5514,
 'eval_samples_per_second': 166.387,
 'eval_steps_per_second': 5.281,
 'epoch': 2.9968782518210197}

In [ ]:
trainer.save_model("ner_model")

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("ner_model")
tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")

In [ ]:
def ner_tag_transformer(text):
    pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="max")
    return pipe(text)

In [ ]:
from transformers import pipeline

In [ ]:
ner_tag_transformer("CHARLES SCHWAB & CO., INC.")

[{'entity_group': 'LABEL_0',
  'score': 0.99999064,
  'word': 'CHARLES SCHWAB & CO.,',
  'start': 0,
  'end': 21},
 {'entity_group': 'LABEL_1',
  'score': 0.99998784,
  'word': 'INC.',
  'start': 22,
  'end': 26}]

In [ ]:
import pickle

# Save the trained model to a pickle file
with open('trained_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# # Load the trained model from the pickle file
# with open('trained_model.pkl', 'rb') as f:
#     model = pickle.load(f)
